# 10.1 GroupBy Mechanics

In [2]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'key1':['a','a','b','b','a'],
                    'key2':['one','two','one','two','one'],
                    'data1':np.random.randn(5),
                    'data2':np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.223267,-0.937650
1,a,two,-1.031859,-0.412718
2,b,one,-1.713260,1.674444
3,b,two,1.581162,-0.575639
4,a,one,-0.957233,-0.861112


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a   -0.588608
b   -0.066049
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.366983
      two    -1.031859
b     one    -1.713260
      two     1.581162
Name: data1, dtype: float64

In [6]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])
years = np.array([2005,2005,2006,2005,2006])

df['data1'].groupby([states, years]).mean()

California  2005   -1.031859
            2006   -1.713260
Ohio        2005    0.902214
            2006   -0.957233
Name: data1, dtype: float64

In [7]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.588608,-0.737160
b,-0.066049,0.549403


In [8]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.366983 -0.899381
     two  -1.031859 -0.412718
b    one  -1.713260  1.674444
     two   1.581162 -0.575639

In [9]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## 10.1.1 Iterating Over Groups

## 10.1.2 Selecting a Column or Subset of Columns

## 10.1.3 Grouping with Dicts and Series

## 10.1.4 Grouping with Functions

## 10.1.5 Grouping by Index Levels

# 10.2 Data Aggregation

## 10.2.1 Column-Wise and Multiple Function Application

## 10.2.2 Returning Aggregated Data Without Row Indexes 

# 10.3 Apply: General split-apply-combine

## 10.3.1 Suppressing the Group Keys

## 10.3.2 Quantile and Bucket Analysis

## 10.3.3 Example: Filling Missing Values with Group-Specific Values

## 10.3.4 Example: Random Sampling and Permutation

## 10.3.5 Example: Group Weighted Average and Correlation

## 10.3.6 Example: Group-Wise Linear Regression 

# 10.4 Pivot Tables and Cross-Tabulation

## 10.4.1 Cross-Tabulations: Crosstab

# 10.5 Conclusion